In [1]:
import time
import tqdm
import csv
import tqdm

In [2]:
csv.field_size_limit(100000000)
csvpath = "data/all_sampled2.csv"
num_articles = 1e10

news = []

with open(csvpath) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    t0 = time.time()
    for i, row in enumerate(csv_reader):
        # if i%1000==0:
        #   print(i)
        if i>=num_articles:
            break
        article = ",".join(row[7:-3])
        date = row[2].strip().split(' ')[0].replace('-', '')[1:7]+"01"
        source = row[-1].split("'")[1]
        link = row[-3]
        
        news_source_dict = {}
        news_source_dict['date'] = date
        news_source_dict['news_source'] = source
        news_source_dict['url'] = link
        
        news.append(news_source_dict)
        # pred = learn.predict(article)
        # results.append(int(pred[1]))

    t1 = time.time() - t0
    print(t1)


12.71827507019043


In [3]:
preds = open("data/preds.txt").read().split('\n')[:-1]
preds_ = list(map(lambda x: x.split(' ')[1], preds))
# print(preds_)

for i, elem in enumerate(news):
    if i>=len(preds_):
        break
    elem['pred_label'] = preds_[i]

news = news[:len(preds_)]

news_dict = {}
for elem in news:
    key = elem['news_source']
#     print(key, elem)
#     break
    if key in news_dict:
        news_dict[key].append(elem)
    else:
        news_dict[key] = [elem]

In [4]:
count=0
news_dict2 = {}
for key in news_dict:
    num = len(news_dict[key])
    count+=num
    if num>=9000:
        news_dict2[key] = news_dict[key]


print(len(news_dict2))

count=0
for key in news_dict2:
    count+=len(news_dict2[key])
    num = len(news_dict2[key])
    print(key, num)

print(count)

news_dict2["Reuters"][0]

8
Reuters 76061
The New York Times 21913
People 14364
CNBC 16165
Vice 10348
The Hill 18714
CNN 9558
Mashable 9072
176195


{'date': '20170801',
 'news_source': 'Reuters',
 'url': " 'https://www.reuters.com/article/brief-ve-wong-to-pay-2016-dividend-on-oc/brief-ve-wong-to-pay-2016-dividend-on-oct-5-idUSL4N1KX33L'",
 'pred_label': 'TECH'}

In [23]:
news_dict3 = {}
# for key in news_dict2:
#     news_dict3[key] = {}
    
for key in news_dict2:
#     years = [elem['date'][:4] for elem in news_dict2[key]]
    row = {}
    for elem in news_dict2[key]:
        year = elem['date'][:4]
        if year in row:
            row[year].append(elem)
        else:
            row[year] = [elem]
        
    news_dict3[key] = row
#     print(len(row['2019']))


count = 0
for key in news_dict3:
    print(key)
    for key2 in news_dict3[key]:
        print(key2, len(news_dict3[key][key2]))

print(news_dict3['Reuters']['2016'][100])


Reuters
2017 23260
2016 20722
2018 17910
2019 14169
The New York Times
2019 3029
2018 5739
2016 7328
2017 5817
People
2018 3812
2019 3045
2016 3632
2017 3875
CNBC
2018 3789
2017 4049
2019 5140
2016 3187
Vice
2019 1003
2016 4013
2017 3294
2018 2038
The Hill
2019 3882
2017 5169
2016 4618
2018 5045
CNN
2017 2365
2018 2750
2019 2275
2016 2168
Mashable
2016 3182
2018 1787
2017 2775
2019 1328
{'date': '20160101', 'news_source': 'Reuters', 'url': " 'http://www.reuters.com/article/us-opec-ecuador-idUSKCN0V32KJ'", 'pred_label': 'ENVIRONMENT'}


In [42]:
news_dict4 = {}
for key in news_dict3:
    news_dict4[key] = {}
    for key2 in news_dict3[key]:
        row = {}
        for elem in news_dict3[key][key2]:
            label = elem["pred_label"]
            if label in row:
                row[label].append(elem)
            else:
                row[label] = [elem]
        news_dict4[key][key2] = row

category_list = []
count = 0
for key in news_dict4:
    for key2 in news_dict4[key]:
        for key3 in news_dict4[key][key2]:
            elem = news_dict4[key][key2]
            num = len(news_dict4[key][key2][key3])
#             print(key3, num)
            category_list.append(key3)
            count+=num
#         break
#     break
category_list = list(set(category_list))

In [67]:
csv_template = ""
for _ in range(len(category_list)+2):
    csv_template+='{},'
csv_template+='\n'

with open("data/bardata_real.csv", 'w') as outfile:
    headers = csv_template.format("News-source", "year", *category_list)
    outfile.write(headers)
    for key in news_dict4:
        for key2 in news_dict4[key]:
            res = []
            for key3 in category_list:
                if key3 in news_dict4[key][key2]:
                    num = len(news_dict4[key][key2][key3])
                else:
                    num = 0
                res.append(num)
            write_res = csv_template.format(key, key2, *res)
            outfile.write(write_res)